# Goals:
-	Build a model to predict the number of sales (Item_Outlet_Sales) using the available features.
-	Gain insights about the number of sales.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
%matplotlib inline

# Overview

In [ ]:
# read the train data
data = pd.read_csv('Data\BigMart.csv')
# check for the null values
data.isna().sum()
# Drop row number 
data.drop("Unnamed: 0", axis = 1, inplace=True) 


In [ ]:
# data.Item_Weight.hist()
data.Outlet_Size.hist()

In [ ]:
data.describe()
df_ = data.select_dtypes(exclude=['int', 'float'])
for col in df_.columns:
    print(df_[col].unique()) # to print categories name only
    # print(df_[col].value_counts()) # to print count of every category

## Data clean

In [ ]:
# Fix some irregular values 
data.loc[data['Item_Fat_Content'].isin(['LF','low fat']), 'Item_Fat_Content'] = 'Low Fat'
data.loc[data['Item_Fat_Content'].isin(['reg']), 'Item_Fat_Content'] = 'Regular'

In [ ]:
# impute NAs by mean
data.Item_Weight.fillna(data.Item_Weight.mean(),inplace=True)
# during EDA in R, discovered NAs present in Grocery and Supermarket type 1. 
# assume Grocery store are small and use mode for Type1 supermarket 

data.loc[:, ['Outlet_Type', 'Outlet_Size']].drop_duplicates()
data.loc[data['Outlet_Type'].eq('Grocery Store') & data['Outlet_Size'].isna(), 'Outlet_Size'] = 'Small'
data.Outlet_Size.mode()
data.loc[data['Outlet_Type'].eq('Supermarket Type1') & data['Outlet_Size'].isna(), 'Outlet_Size'] = 'Medium'
data.loc[:, ['Outlet_Type', 'Outlet_Size']].drop_duplicates()
data.loc[:, ['Item_Type']].drop_duplicates()
data.Outlet_Location_Type = data.Outlet_Location_Type.astype('str')
# save it 
data.to_csv("Data/cleaned.csv", index=False)


In [ ]:
data = pd.read_csv('Data/cleaned.csv')

## Data preProcessing 

### Code categorical variables 

In [ ]:
# create an object of the OneHotEncoder
Encoder = ce.OneHotEncoder(cols=['Item_Fat_Content',
                                 'Item_Type',
                                 'Outlet_Identifier',
                                 'Outlet_Size',
                                 'Outlet_Location_Type',
                                 'Outlet_Type'],
                                 use_cat_names=True)
# encode the categorical variables
data = Encoder.fit_transform(data)

In [ ]:
# general information about the data
data.isna().sum()
data.describe()
data



### Standarise numeric variables 

In [ ]:
# create the StandardScaler 
scaler = StandardScaler()
# fit the Item_MRP and Weigh, and visibility 
num_cols = ['Item_Weight','Item_Visibility','Item_MRP']
# fit the Item_MRP and Weight and visibility
data[num_cols] = scaler.fit_transform(data[num_cols])


### Split data to train and test 

In [ ]:
# separate the predictors and target variable 
train_X = data.drop(columns=['Item_Identifier','Item_Outlet_Sales'])
train_Y = data['Item_Outlet_Sales']

# randomly split the data
train_x, test_x, train_y, test_y = train_test_split(train_X, train_Y,test_size=0.2,random_state=0)

# shape of train and test splits
train_x.shape, test_x.shape, train_y.shape, test_y.shape

## Baseline model

In [ ]:
# create an object of the LinearRegression Model
model_LR = linear_model.LinearRegression(fit_intercept=False)

# fit the model with the training data
model_LR.fit(train_x, train_y)

# predict the target on train and test data 
predict_train = model_LR.predict(train_x)
predict_test  = model_LR.predict(test_x)

print('RMSE on train data: ', mean_squared_error(train_y, predict_train))
print('RMSE on test data: ',  mean_squared_error(test_y, predict_test))
print('MAE on train data: ', mean_absolute_error(train_y, predict_train))
print('MAE on test data: ',  mean_absolute_error(test_y, predict_test))
print('r2_score on train data: ', r2_score(train_y, predict_train))
print('r2_score on test data: ',  r2_score(test_y, predict_test))
plt.figure(figsize=(18,6))
plt.subplot(1, 2, 1)
plt.scatter(train_y,predict_train)
# plt.title( 'Link function ' + str(i) + '(Train data)')
plt.subplot(1, 2, 2)
plt.scatter(test_y,predict_test)
# plt.title( 'Link function ' + str(i) + '(Test data)')

In [ ]:
## Extract feature coefs
d = {'featurename':model_LR.feature_names_in_, 
     'featurecoef':model_LR.coef_}
# coef = pd.Series(model_LR.coef_, model_LR.feature_names_in_, name = 'value').sort_values()

pd.DataFrame(d)


In [ ]:
## Output evaluation results 
pd.DataFrame({'obs':test_y, 'pred':predict_test}).to_csv('Data/lmtestrs.csv', index = False)

### Try a random forest

In [ ]:
# create an object of the RandomForestRegressor
model_RFR = RandomForestRegressor(max_depth=10)

# fit the model with the training data
model_RFR.fit(train_x, train_y)

# predict the target on train and test data
predict_train = model_RFR.predict(train_x)
predict_test = model_RFR.predict(test_x)

print('RMSE on train data: ', mean_squared_error(train_y, predict_train))
print('RMSE on test data: ',  mean_squared_error(test_y, predict_test))
print('MAE on train data: ', mean_absolute_error(train_y, predict_train))
print('MAE on test data: ',  mean_absolute_error(test_y, predict_test))
print('r2_score on train data: ', r2_score(train_y, predict_train))
print('r2_score on test data: ',  r2_score(test_y, predict_test))

# RF is better than LR so far. But overfitting 

# Try a lasso regression to drop some features?

In [ ]:
from sklearn import linear_model
regressor = linear_model.Lasso(alpha=100,  
                               positive=True,
                               fit_intercept=False, 
                               max_iter=1000,
                               tol=0.0001)
regressor.fit(train_x, train_y)

In [ ]:
import eli5
eli5.show_weights(regressor, top=-1, feature_names = train_x.columns.tolist())


In [ ]:
df_L1=pd.DataFrame({'name': regressor.feature_names_in_,
              'coefs': regressor.coef_})
df_L1[df_L1['coefs' ]> 0 ].sort_values(by='coefs', ascending=False)


## Five features lead the way? 
Not sure if it's going to work

In [ ]:
# Respilt the data
train_X = data.loc[:,df_L1[df_L1['coefs' ]> 0 ].name.tolist()]
train_Y = data['Item_Outlet_Sales']

# split the data
train_x, test_x, train_y, test_y = train_test_split(train_X, train_Y,test_size=0.2,random_state=0)

# shape of train and test splits
train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [ ]:
# create the LinearRegression Model
model_LR = linear_model.LinearRegression()

# fit the model with the training data
model_LR.fit(train_x, train_y)

# predict the target on train and test data 
predict_train = model_LR.predict(train_x)
predict_test  = model_LR.predict(test_x)

# Root Mean Squared Error on train and test date
print('RMSE on train data: ', mean_squared_error(train_y, predict_train))
print('RMSE on test data: ',  mean_squared_error(test_y, predict_test))
print('MAE on train data: ', mean_absolute_error(train_y, predict_train))
print('MAE on test data: ',  mean_absolute_error(test_y, predict_test))
print('r2_score on train data: ', r2_score(train_y, predict_train))
print('r2_score on test data: ',  r2_score(test_y, predict_test))

In [ ]:
# create the RandomForestRegressor
model_RFR = RandomForestRegressor(max_depth=10)

# fit the model with the training data
model_RFR.fit(train_x, train_y)

# predict the target on train and test data
predict_train = model_RFR.predict(train_x)
predict_test = model_RFR.predict(test_x)

# Root Mean Squared Error on train and test data
print('RMSE on train data: ', mean_squared_error(train_y, predict_train)**(0.5))
print('RMSE on test data: ',  mean_squared_error(test_y, predict_test)**(0.5))
print('MAE on train data: ', mean_absolute_error(train_y, predict_train))
print('MAE on test data: ',  mean_absolute_error(test_y, predict_test))
print('r2_score on train data: ', r2_score(train_y, predict_train))
print('r2_score on test data: ',  r2_score(test_y, predict_test))

### Not doing great
Need a pipleline and see more models


## Model selection 

### Prepara data 

In [ ]:
# read the train data
data = pd.read_csv('Data/cleaned.csv')
Encoder = ce.OneHotEncoder(cols=['Item_Fat_Content',
                                 'Item_Type',
                                'Outlet_Identifier',
                                'Outlet_Size',
                                'Outlet_Location_Type',
                                'Outlet_Type'],use_cat_names=True)
# encode the categorical variables
data = Encoder.fit_transform(data)
num_cols = ['Item_Weight','Item_Visibility','Item_MRP']
scaler = StandardScaler()
# fit the Item_MRP and Weight
data[num_cols] = scaler.fit_transform(data[num_cols])

# define a fun to do col selections and split 
def split_traintest(df, dropvals, yvals = 'Item_Outlet_Sales'):

    # separate the predictors and target variable 
    X = df.drop(columns=dropvals)
    Y = data[yvals]

    # randomly split the data
    train_x, test_x, train_y, test_y = train_test_split(X, Y,test_size=0.2,random_state=42)
    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_traintest(data, ['Item_Identifier','Item_Outlet_Sales'])
# check shape of train and test splits
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)


### Try six with 10 fold 

In [ ]:
# baselines
num_folds = 10
seed = 7

models = []
models.append(('LM', linear_model.LinearRegression()))
models.append(('L1', linear_model.Lasso()))
models.append(('L2', linear_model.Ridge()))
models.append(('BayesRidge',linear_model.BayesianRidge()))
models.append(('GLM',linear_model.TweedieRegressor(link='log',  max_iter=3000)))
models.append(('RF', RandomForestRegressor(max_depth=10, random_state=0)))
# build a pipeline
results = []
names = []
# copied from  https://www.kaggle.com/richarde/easy-pipeline-and-model-selection#2.0-Process-the-Data
for name, model in models:
    kfold = KFold(n_splits=num_folds)
    cv_results = cross_val_score(model, train_x, train_y,  scoring = 'r2',cv=kfold)
    results.append(cv_results)
    names.append(name)
    msg = "%s %f %f " % (name, cv_results.mean(), cv_results.std())
    print(msg)

### Visualisation 

In [ ]:
# compare algorithms
fig = plt.figure()
fig.suptitle('Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()
plt.savefig('Reports/baselines.png', dpi = 300)

In [ ]:
## save output for visualisation in R
rs_hold = [item for sublist in results for item in sublist]
df = pd.DataFrame({'names' :np.repeat(['LM','L1','L2','BayeL2', 'GLM', 'RF'], 10),
                   'r2':rs_hold})
## output 
df.to_csv('Data/baselines.csv')

## Hyperparameter tuning 

### GLM

In [ ]:
# create an object of the RandomForestRegressor
iterations = [5000]
links = ['log']
coefs = []
results = []
names = []
for i in links:
    model_glm = linear_model.TweedieRegressor( link=i,   max_iter=5000)
    # fit the model with the training data
    model_glm.fit(train_x, train_y)
    # predict the target on train and test data
    predict_train = model_glm.predict(train_x)
    predict_test = model_glm.predict(test_x)  
    # get the coef
    coef = pd.Series(model_glm.coef_, model_glm.feature_names_in_, name = 'value').sort_values()
    coefs.append(coef)    

    print('r2_score on train data: ', r2_score(train_y, predict_train))
    print('r2_score on test data: ',  r2_score(test_y, predict_test))    
    plt.figure(figsize=(18,6))
    plt.subplot(1, 3, 1)
    plt.scatter(train_y,predict_train)
    plt.title( 'Link function ' + str(i) + '(Train data)')
    plt.subplot(1, 3, 2)
    plt.scatter(test_y,predict_test)
    plt.title( 'Link function ' + str(i) + '(Test data)')
    plt.subplot(1, 3, 3)
    coef[coef>0.1].plot(kind='bar', title='Model Coefficients')
    
    plt.show()

## Save coefs 
pd.DataFrame(coefs).to_csv('Data/glm.csv', index = False)

### RF

In [ ]:
# create an RandomForestRegressor
param_grid = {'bootstrap': [True],
              'max_depth': [5, 10, None], 
              'max_features': ['auto', 'log2'], 
              'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]
              }
              
g_search = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid, 
                        cv = 5, n_jobs = 1, verbose = 0,
                        return_train_score=True)
g_search.fit(train_x, train_y)
## save output 
df_grid = pd.concat([pd.DataFrame(g_search.cv_results_["params"]),
                     pd.DataFrame(g_search.cv_results_["mean_test_score"], 
                                  columns=["mean_test_score"]),
                     pd.DataFrame(g_search.cv_results_["rank_test_score"], 
                                  columns=["rank_test_score"])],axis=1)
df_grid.to_csv("Data/rf_gridsearch.csv", index=False)

In [ ]:

model_rf = RandomForestRegressor( bootstrap=True, max_depth=5, max_features='auto', n_estimators=13, random_state=42) 
# fit the model with the training data
model_rf.fit(train_x, train_y)
# predict the target on train and test data
predict_train = model_rf.predict(train_x)
predict_test = model_rf.predict(test_x)  
# get the coef
# coef = pd.Series(model_rf.coef_, model_rf.feature_names_in_, name = 'value').sort_values()
# coefs.append(coef)

print('r2_score on train data: ', r2_score(train_y, predict_train))
print('r2_score on test data: ',  r2_score(test_y, predict_test))   

In [ ]:
## best performer 
feature_impt = pd.Series(model_rf.feature_importances_, model_rf.feature_names_in_, name = 'value').sort_values()
feature_impt[feature_impt>0.01].plot(kind = 'bar')
## save output 
pd.DataFrame(feature_impt[feature_impt>0]).to_csv('Data/rf_featureimpt.csv')

## Plotting some useful figures 

In [ ]:
# separate the predictors and target variable 
train_X = data.drop(columns=['Item_Identifier','Item_Outlet_Sales'])
train_Y = data['Item_Outlet_Sales']

# randomly split the data
train_x, test_x, train_y, test_y = train_test_split(train_X, train_Y,test_size=0.2,random_state=0)

# shape of train and test splits
train_x.shape, test_x.shape, train_y.shape, test_y.shape


# Try plot a learning curve
from sklearn.linear_model import RidgeCV
from yellowbrick.model_selection import LearningCurve

# Instantiate the regression model and visualizer
model = linear_model.LinearRegression()
visualizer = LearningCurve(model, scoring='r2')

visualizer.fit(train_X, train_Y)        # Fit the data to the visualizer
visualizer.show('Reports/lmlearningcurve.png')           # Finalize and render the figure




In [ ]:
from yellowbrick.model_selection import ValidationCurve
viz = ValidationCurve(
    RandomForestRegressor(), param_name="max_depth",
    param_range=np.arange(1, 11), cv=10, scoring="r2"
)

# Fit and show the visualizer
viz.fit(train_X, train_Y)
viz.show(outpath='Reports/rf_maxdepth.png')

In [ ]:

from yellowbrick.model_selection import LearningCurve

model = linear_model.TweedieRegressor(link='log',   max_iter=3000)
visualizer = LearningCurve(model, scoring='r2')

visualizer.fit(train_x, train_y)        # Fit the data to the visualizer
visualizer.show(outpath= 'Reports/glmLearningCurve.png')



In [ ]:
# Compute train and test errors
from sklearn import linear_model
alphas = np.logspace(-5, 1, 10)
enet = linear_model.ElasticNet(l1_ratio=0.7, max_iter=10000)
train_errors = list()
test_errors = list()
for alpha in alphas:
    enet.set_params(alpha=alpha)
    enet.fit(train_x, train_y)
    train_errors.append(enet.score(train_x, train_y))
    test_errors.append(enet.score(test_x, test_y))

i_alpha_optim = np.argmax(test_errors)
alpha_optim = alphas[i_alpha_optim]
print("Optimal regularization parameter : %s" % alpha_optim)

# Estimate the coef_ on full data with optimal regularization parameter
enet.set_params(alpha=alpha_optim)
coef_ = enet.fit(train_x, train_y).coef_

In [ ]:
plt.subplot(2, 1, 1)
plt.semilogx(alphas, train_errors, label="Train")
plt.semilogx(alphas, test_errors, label="Test")
plt.vlines(
    alpha_optim,
    plt.ylim()[0],
    np.max(test_errors),
    color="k",
    linewidth=3,
    label="Optimum on test",
)
plt.legend(loc="lower left")
plt.ylim([0, 1.2])
plt.xlabel("Regularization parameter")
plt.ylabel("Performance")

# Show estimated coef_ vs true coef
plt.subplot(2, 1, 2)

plt.plot(coef_, label="Estimated coef")
plt.legend()
plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.26)
plt.show()

In [ ]:
# some feature enigeering change supermarket type 2 to 1

# data.loc[data['Item_Type'].isin(['Household', 'Health and Hygiene']), 'Item_Type'] = 'Noneedible'
# data.loc[data['Outlet_Type'].eq('Supermarket Type2'), 'Outlet_Type'] = 'Supermarket Type1'